https://console.groq.com/docs/tool-use

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [2]:
from groq import Groq
import json

client = Groq()
# MODEL = 'llama3-groq-70b-8192-tool-use-preview'
MODEL="llama-3.1-70b-versatile" 

# Tools

In [3]:
def calculate(expression):
    """Evaluate a mathematical expression"""
    try:
        result = eval(expression)
        return json.dumps({"result": result})
    except:
        return json.dumps({"error": "Invalid expression"})

In [4]:
def get_messages(user_prompt):

    messages=[
        {
            "role": "system",
            "content": "You are a tool assistant. Use any of the provided tools and return the results."
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]
    return messages


In [5]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "calculate",
            "description": "Evaluate a mathematical expression",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "The mathematical expression to evaluate",
                    }
                },
                "required": ["expression"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_simulation_list",
            "description": "return a list of all the simulation names",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    }
]

In [6]:
user_prompt = "What is 25 * 4 + 10?"

# user_prompt = "what is NVIDIA Air?"

# user_prompt = "show me the simulation list"

In [7]:
messages = get_messages(user_prompt)

response = client.chat.completions.create(
    model=MODEL,
    messages=messages,
    tools=tools,
    tool_choice="auto",
    max_tokens=4096,
    temperature=1e-8 
)

In [8]:
response

ChatCompletion(id='chatcmpl-4f9466ad-e6f0-4d08-bcdc-8101e1856a6e', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_wm6k', function=Function(arguments='{"expression": "25 * 4 + 10"}', name='calculate'), type='function')]))], created=1724740376, model='llama-3.1-70b-versatile', object='chat.completion', system_fingerprint='fp_b3ae7e594e', usage=CompletionUsage(completion_tokens=20, prompt_tokens=293, total_tokens=313, completion_time=0.081375365, prompt_time=0.08465596, queue_time=0.25411857400000004, total_time=0.166031325), x_groq={'id': 'req_01j698k5xke5xbb4rmxtyddhgf'})

In [9]:
len(response.choices)

1

In [10]:
response_message = response.choices[0].message
response_message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_wm6k', function=Function(arguments='{"expression": "25 * 4 + 10"}', name='calculate'), type='function')])

In [11]:
tool_calls = response_message.tool_calls
tool_calls

[ChatCompletionMessageToolCall(id='call_wm6k', function=Function(arguments='{"expression": "25 * 4 + 10"}', name='calculate'), type='function')]

In [12]:
available_functions = {
    "calculate": calculate,
}
messages.append(response_message)

In [13]:
tool_call = tool_calls[0]
function_name = tool_call.function.name
function_to_call = available_functions[function_name]
function_args = json.loads(tool_call.function.arguments)
print(function_name,", ", function_to_call,", ", function_args)



calculate ,  <function calculate at 0x7f1f9f691e10> ,  {'expression': '25 * 4 + 10'}


In [14]:
function_response = function_to_call(
    expression=function_args.get("expression")
)
print(function_response)

{"result": 110}


In [15]:
messages 

[{'role': 'system',
  'content': 'You are a tool assistant. Use any of the provided tools and return the results.'},
 {'role': 'user', 'content': 'What is 25 * 4 + 10?'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_wm6k', function=Function(arguments='{"expression": "25 * 4 + 10"}', name='calculate'), type='function')])]

In [ ]:
messages.append(
    {
        "role": "tool",
        "tool_call_id": "call_ckcb",
        "name": "get_simulation_list",
        "content": '["Sim1"]',
    }
)

In [ ]:
messages

In [ ]:
messages.append(
    {
        "tool_call_id": tool_call.id,
        "role": "tool",
        "name": function_name,
        "content": function_response,
    }
)

In [ ]:

second_response = client.chat.completions.create(
    model=MODEL,
    messages=messages,
    # temperature=1e-8 
)
content = second_response.choices[0].message.content
print(content)

## all in a loop

In [ ]:
if tool_calls:
    available_functions = {
        "calculate": calculate,
    }
    messages.append(response_message)
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            expression=function_args.get("expression")
        )
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )
    second_response = client.chat.completions.create(
        model=MODEL,
        messages=messages
    )
    content = second_response.choices[0].message.content
    print(content)
else:
    print("no tool found")    

# Manual Query

In [12]:
messages = [{
        "role": "system",
        "content": """
                You are a profitional technical assistant that helps the user build a network simulation using NVIDIA AIR.
                NVIDIA Air is a cloud-hosted data center simulation platform designed to create a virtual environment 
                that mimics real-world production settings. It allows users to build and run simulations of 
                complex data center networks, enabling them to validate configurations, features, and automation code effectively. 
                You should answer questions related to this topic only. Be consice in your answers.
                Do not offer to do anything that you do not have a tool for. 
                """
    }, {
        "role": "user",
        "content": "show me the simulation list"
    }, {
        "role": "assistant",
        "tool_calls": [{
                "id": "call_ckcb",
                "function": {
                    "name": "get_simulation_list",
                    "arguments": "{}"
                },
                "type": "function"
            }
        ]
    }, {
        "role": "tool",
        "tool_call_id": "call_ckcb",
        "name": "get_simulation_list",
        "content": "[]"
    }
]

In [13]:
second_response = client.chat.completions.create(
    model=MODEL,
    messages=messages,
    # temperature=1e-8 
)
content = second_response.choices[0].message.content
print(content)

You have no simulations created. 

Simulation list is currently empty.


In [19]:
second_response

ChatCompletion(id='chatcmpl-64dc8361-f895-45e5-9e54-cafb6f6b6045', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="The simulation list is ['Sim1'].", role='assistant', function_call=None, tool_calls=None))], created=1724740467, model='llama-3.1-70b-versatile', object='chat.completion', system_fingerprint='fp_5c5d1b5cfb', usage=CompletionUsage(completion_tokens=9, prompt_tokens=81, total_tokens=90, completion_time=0.036203285, prompt_time=0.024573227, queue_time=7.519046786, total_time=0.060776512), x_groq={'id': 'req_01j698nyz7e65b8f7rrfh8bw6r'})